In [1]:
#Gathering Data manually from google

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#path to directory with images-- /content/drive/MyDrive/Python and DS/Minor Project/images

In [4]:
#preprocessing data

import os 
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize

target = []
images = []
flat_data = []

DATADIR = "/content/drive/MyDrive/Python and DS/Minor Project/images"
CATEGORIES = ["rose","tree"]
for category in CATEGORIES:
  class_num = CATEGORIES.index(category)
  path = os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    #plt.imshow(img_array)
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    target.append(class_num)

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.2,random_state=42, stratify=target)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm 
param_grid = [
           
          {'C':[1,10,100,1000],'kernel': ['linear']},
          {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']}
           
]

svc = svm.SVC(probability=True)
model = GridSearchCV(svc,param_grid)
model.fit(x_train,y_train)

In [ ]:
y_pred = model.predict(x_test)
y_pred

In [ ]:
y_test

In [9]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
accuracy_score(y_pred , y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
import joblib 
joblib.dump(model,"img_model")

In [ ]:
!pip install streamlit -- quiet 
!pip install pyngrok --quiet
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
import numpy as np
from PIL import Image
st.set_option("deprecation.showfileUploaderEncoding",False)
st.title("Cat and Dog classifier")
st.text("Upload the Image")

model = joblib.load("img_model")

url = input("Enter your URL")
img1 = imread(url)
if img1 is not None:
  img = Image.open(img1)
  st.image(img,caption = "Image URL")

  if st.button("Predict"):
    CATEGORIES = ["rose","tree"]
    st.write("Result..")
    flat_data=[]
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    st.write(img.shape)
    y_out = model.predict(flat_data)
    y_out = CATEGORIES[y_out[0]]
    st.title(f"Predicted Output: {y_out}")
    q = model.predict_proba(flat_data)
    for index,item in enumerate(CATEGORIES):
      st.write(f"{item} : {q[0][index]*100}%")


In [ ]:
!nohup streamlit run app.py &
url = ngrok.connect(port="8501")
url

In [ ]:
!pip install pipreqs
!pipreqs /content